### Методы

In [10]:
class Point:
    color = 'red'
    circle = 2

    # self - ссылка на экземпляр класса, позволяет работать с локальными атрибутами конкретного экземпляра
    def set_coords(self):
        print("Вызов метода set_coords" + str(self))

In [2]:
Point.set_coords

<function __main__.Point.set_coords()>

In [3]:
Point.set_coords()

Вызов метода set_coords


In [11]:
pt = Point()

In [12]:
pt.set_coords()

Вызов метода set_coords<__main__.Point object at 0x11078f5e0>


In [36]:
class Point:
    color = 'red'
    circle = 2
    # инициализатор класса
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    # финализатор класса
    def __del__(self):
        print("Удаление экземпляра:" + str(self))
        
    # self - ссылка на экземпляр класса
    def set_coords(self, x, y):
        self.x = x
        self.y = y

    def get_coords(self):
        return (self.x, self.y)

In [27]:
pt = Point()
pt.set_coords(1, 2)
print(pt.__dict__)

{'x': 1, 'y': 2}


In [28]:
pt2 = Point()
pt2.set_coords(10, 20)
print(pt2.__dict__)

{'x': 10, 'y': 20}


In [29]:
pt.get_coords()

(1, 2)

Методы также являются атрибутами классов

In [30]:
f = getattr(pt, 'get_coords')

In [31]:
f()

(1, 2)

In [34]:
new_pt = Point()
print(new_pt.__dict__)

{'x': 0, 'y': 0}


- Магический метод _ _ new _ _ вызывается перед созданием класса
- Магический метод _ _ init _ _ вызывается сразу после создания объекта класса. 

In [ ]:
# cls ссылается на класс (Point), а self - на созданный экземпляр
class Point:
    def __new__(cls, *args, **kwargs):
        print("Вызов __new__ для" + str(cls))
        return super().__new__(cls)
    
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

In [ ]:
pt = Point(1, 2)
print(pt)

Вызов __new__ для<class '__main__.Point'>
None


Магический метод new запускает создание экземпляра класса и возвращает адрес нового объекта: необходимо из базового класса super() вызвать магический метод new


Паттерн Singleton. В программе должен существовать только один экземпляр класса.

In [ ]:
class DataBase:

    # новый атрибут, instance - ссылка на экземпляр класса
    __instance = None

    def __new__(cls, *args, **kwargs):
        # проверяем атрибут 
        if cls.__instance is None:
            cls.__instance = super().__new__(cls)
        return cls.__instance

    def __del__(self):
        DataBase.__instance = None

    def __init__(self, user, psw, port):
        self.user = user
        self.psw = psw
        self.port = port
    
    def connect(self):
        print(f"Соединение с БД {self.user}, {self.psw}, {self.port}")

    def read(self):
        return "Данные из БД"

    def close(self):
        print("Закрытие соединения") 

    def write(self, data):
        print(f"Запись в БД {data}")

In [ ]:
db = DataBase('root', '1234', 80)
db2 = DataBase('root2', 'ty6789', 40)

In [ ]:
print(id(db), id(db2))

140292280680608 140292280680608


In [ ]:
db.connect()
db2.connect()

Соединение с БД root2, ty6789, 40
Соединение с БД root2, ty6789, 40


Инициализация происходит дважды и локальные значения изменяются. Для того, чтобы это исправить, необходимо вызвать магический метод _ _ call _ _ 

### Методы классов и статические методы

In [1]:
class Vector:
    MIN_COORD = 0
    MAX_COORD = 100

    @classmethod
    def validate(cls,args):
        return cls.MIN_COORD <= args <= cls.MAX_COORD

    def __init__(self, x, y):
        self.x = self.y = 0
        if self.validate(x) and self.validate(y):
            self.x = x
            self.y = y

    def get_coord(self):
        return self.x, self.y

    @staticmethod
    def norm2(x, y):
        return x*x + y*y


In [3]:
v = Vector(5,5)
v.get_coord()

(5, 5)

In [4]:
Vector.get_coord(v)

(5, 5)

v.get_coord() <=> Vector.get_coord(v)

В методах классов используется ссылка на сам класс, в обычных методах - на объект класса, статические методы являются независимыми и не имеют ссылок ни на сам класс, ни на эккземпляр, но при этом логически связаны с самим классом.

## Механизм инкапсуляции

In [1]:
class Point:
    def __init__(self, x=0, y=0):
        self.x, self.y = x, y

In [2]:
pt = Point(1, 2)

In [5]:
pt.x = 100
pt.y = 200

In [6]:
print(pt.x, pt.y)

100 200


In [3]:
pt.x = 'coord'

Если мы не хотим, чтобы пользователь напрямую имел доступ к локальным свойствам экземпляров класса, то необходимо помечать их закрытыми.


- attribute (без одного или двух подчёркиваний вначале) - публичное свойство
- _ attribute (с одним подчёркиванием) - режим доступа protected (служит для обращения внутри класса и во всех его дочерних классах)
- _ _ attribute (с двумя подчёркиваниями) - режим доступа private (служит для обращения только внутри класса)

#### Protected режим доступа

In [4]:
class Point:
    def __init__(self, x=0, y=0):
        self._x, self._y = x, y

pt = Point(100, 100)

In [10]:
# режим доступа protected лишь сигнал для программиста, что лучше не обращаться к атрибуту напрямую
print(pt._x, pt._y)

100 100


#### Private режим доступа + интерфейсные методы (сеттеры и геттеры)

In [30]:
class Point:
    def __init__(self, x=0, y=0):
        self.__x = self.__y = 0
        if self.__check_value(x) and self.__check_value(y):
            self.__x, self.__y = x, y

     # приватный метод
    @classmethod
    def __check_value(cls, x):
        return type(x) in (int, float)

    # сеттер
    def set_coord(self, x, y):
        if self.__check_value(x) and self.__check_value(y):
            self.__x = x
            self.__y = y
        else:
            raise ValueError("Координаты должны быть числами")
        
    # геттер
    def get_coord(self):
        return self.__x, self.__y


pt = Point(100, 100)

In [16]:
# доступ извне не работает
print(pt.__x, pt.__y)

AttributeError: 'Point' object has no attribute '__x'

In [24]:
# ошибок нет, так как обращение внутри класса
pt.set_coord(10, 20)

In [28]:
# ошибок нет, так как обращение внутри класса
pt.set_coord('10', 20)

ValueError: Координаты должны быть числами

In [22]:
pt.get_coord()

(100, 100)

In [31]:
# Проверка доступности методов для экземпляра
print(dir(pt))

['_Point__check_value', '_Point__x', '_Point__y', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'get_coord', 'set_coord']


In [33]:
# ТАК НЕ ДЕЛАТЬ
print(pt._Point__x)

100


#### Модуль accessify

In [34]:
#pip install accessify

Defaulting to user installation because normal site-packages is not writeable
  Created wheel for accessify: filename=accessify-0.3.1-py3-none-any.whl size=26692 sha256=048fbf540d777bf0363c9d11c0c575a95751d27e93961a9a4ec82862542f6533
  Stored in directory: /Users/masha/Library/Caches/pip/wheels/b7/29/d2/ab2ba3a524556686dff65bd01bfcaeb389f274eb562ee94568
Successfully built accessify
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [35]:
from accessify import private, protected

In [36]:
class Point:
    def __init__(self, x=0, y=0):
        self.__x = self.__y = 0
        if self.check_value(x) and self.check_value(y):
            self.__x, self.__y = x, y

     # приватный метод
    @private
    @classmethod
    def check_value(cls, x):
        return type(x) in (int, float)

    # сеттер
    def set_coord(self, x, y):
        if self.check_value(x) and self.check_value(y):
            self.__x = x
            self.__y = y
        else:
            raise ValueError("Координаты должны быть числами")
        
    # геттер
    def get_coord(self):
        return self.__x, self.__y


pt = Point(100, 100)

In [38]:
# пытаемся обратиться к защищённому методу
pt.check_value(5)

InaccessibleDueToItsProtectionLevelException: Point.check_value() is inaccessible due to its protection level

Защита с модулем accessify надёжнее использования двойного подчёркивания